# Linear Regression Code Along

Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.select('Yearly Amount Spent').show()

+-------------------+
|Yearly Amount Spent|
+-------------------+
|  587.9510539684005|
|  392.2049334443264|
| 487.54750486747207|
|  581.8523440352177|
|  599.4060920457634|
|   637.102447915074|
|  521.5721747578274|
|  549.9041461052942|
|  570.2004089636196|
|  427.1993848953282|
|  492.6060127179966|
|  522.3374046069357|
|  408.6403510726275|
|  573.4158673313865|
|  470.4527333009554|
|  461.7807421962299|
| 457.84769594494855|
| 407.70454754954415|
|  452.3156754800354|
|   605.061038804892|
+-------------------+
only showing top 20 rows



In [ ]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [16]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [11]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [32]:
output = assembler.transform(data)

In [33]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [34]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [35]:
final_data = output.select("features",'Yearly Amount Spent')

In [36]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [37]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                335|
|   mean|  505.9229051357284|
| stddev|  80.38103153784202|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                165|
|   mean| 485.89603581106877|
| stddev|  75.57812701456774|
|    min|  282.4712457199145|
|    max|  662.9610878058954|
+-------+-------------------+



In [39]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [40]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [41]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.222620280677983,38.61983872713692,0.42514860989715964,61.786783532801] Intercept: -1033.8244971815554


In [43]:
test_results = lrModel.evaluate(test_data)

In [44]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  8.331405229545908|
| -12.86356570429541|
|-6.2043770366131525|
|  -5.34272389142501|
| -4.937463266703787|
|-14.188485132581206|
|   -8.6113799678933|
|  9.586217464692936|
|-1.4888485758821162|
| -4.812146290923465|
| 2.6977866576836504|
| -6.554380597762815|
| -9.764577531178418|
|-18.589932056369662|
|-15.335424892871288|
|-1.7600732754279989|
| -2.252860201060173|
| -5.473446087792183|
| -3.101208238826473|
| -9.917595818971165|
+-------------------+
only showing top 20 rows



In [49]:
unlabeled_data = test_data.select('features')

In [50]:
predictions = lrModel.transform(unlabeled_data)

In [51]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 400.3089458430816|
|[30.3931845423455...|  332.792435507489|
|[30.4925366965402...| 288.6756227565277|
|[30.8364326747734...| 472.8446243184146|
|[30.8794843441274...|495.14406325155846|
|[31.0662181616375...|463.12177834025556|
|[31.1280900496166...|  565.864066714948|
|[31.1695067987115...| 417.7703133375999|
|[31.2606468698795...| 422.8154798328335|
|[31.2681042107507...| 428.2826794647474|
|[31.3662121671876...|427.89109589880127|
|[31.5257524169682...| 450.5200074076447|
|[31.5261978982398...| 418.8591037235162|
|[31.5702008293202...| 564.5354241977745|
|[31.5741380228732...| 559.7446970534581|
|[31.6610498227460...|418.11842685532883|
|[31.7216523605090...| 350.0297868329328|
|[31.7656188210424...| 502.0275277233993|
|[31.8186165667690...| 449.5198816089621|
|[31.8279790554652...| 449.9203433659127|
+--------------------+------------

In [52]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.605977158319334
MSE: 92.2747971661528


Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!